# Football Insights & Novel Metrics
Analysis of 'Zone Collapse Speed' and 'Defensive Reaction Time' on Week 1 data.

In [ ]:
import sys
sys.path.append('../src')
from data_loader import DataLoader
from metrics import calculate_zone_collapse_speed, calculate_defensive_reaction_time
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

# Load Data
loader = DataLoader('../')
week1 = loader.load_week_data(1)
plays = loader.load_plays()
week1 = loader.standardize_tracking_directions(week1)

# Filter for passes
pass_plays = plays.filter(pl.col("pass_result").is_in(["C", "I", "S"]))
print(f"Analyzing {pass_plays.height} pass plays.")

## Metric Calculation: Zone Collapse Speed
We calculate how fast the defense contracts after a catch or throw.

In [ ]:
# Pick a play (e.g., first one)
play = pass_plays.row(0, named=True)
game_id = play["game_id"]
play_id = play["play_id"]
def_team = play["defensive_team"]

play_track = week1.filter((pl.col("game_id") == game_id) & (pl.col("play_id") == play_id))

collapse_df = calculate_zone_collapse_speed(play_track, def_team)

plt.figure(figsize=(10, 4))
plt.plot(collapse_df["frame_id"], collapse_df["hull_area_rate"], label="Collapse Rate")
plt.axhline(0, color='r', linestyle='--')
plt.xlabel("Frame")
plt.ylabel("Area Change (sq yds/s)")
plt.title(f"Defensive Collapse Rate: Play {play_id}")
plt.legend()
plt.show()

## Metric Calculation: Reaction Time
How quickly did defenders react after the ball was thrown?

In [ ]:
# Find ball snap/release frame
# events: 'pass_forward', 'ball_snap', etc.
events = play_track.filter(pl.col("event") == "pass_forward")
if events.height > 0:
    release_frame = events["frame_id"][0]
    reaction_time = calculate_defensive_reaction_time(play_track, release_frame, def_team)
    print(f"Defensive Reaction Time: {reaction_time:.2f}s")
else:
    print("No pass_forward event found.")